In [120]:
import pandas as pd
import numpy as np
import time
from scipy.spatial import distance
from scipy.spatial import distance_matrix
from copy import deepcopy

In [146]:
X_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_resampling3000_X.csv")
y_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_resampling3000_y.csv")
X_train=X_train.drop(['robust_dis'],axis=1)
y_train=y_train.drop(['robust_dis'],axis=1)
y_train=y_train.drop(['Unnamed: 0'],axis=1)

In [153]:
train = {'feature0':X_train['feature0'],'feature1':X_train['feature1'],'feature2':X_train['feature2'],'feature3':X_train['feature3'],'feature4':X_train['feature4'],'label':y_train['label'],'indexofpixel':y_train['indexofpixel'],'sourceofpixel':y_train['sourceofpixel']}
train = pd.DataFrame(data=train)
class1_indices=np.random.choice(train[train['label']==1].index,1,replace=False)
class2_indices=np.random.choice(train[train['label']==2].index,1,replace=False)
class1_sample = train.iloc[class1_indices]
class2_sample = train.iloc[class2_indices]
train=train.drop(class1_indices)
train=train.drop(class2_indices)
train = pd.DataFrame.reset_index(train).drop('index',axis=1)
class1_sample = pd.DataFrame.reset_index(class1_sample).drop('index',axis=1)
class2_sample = pd.DataFrame.reset_index(class2_sample).drop('index',axis=1)

In [154]:
Store=list()
newStore=list()
class1_sample= {'feature0':class1_sample.iloc[0]['feature0'],'feature1':class1_sample.iloc[0]['feature1'],'feature2':class1_sample.iloc[0]['feature2'],'feature3':class1_sample.iloc[0]['feature3'],'feature4':class1_sample.iloc[0]['feature4'],'label':class1_sample.iloc[0]['label'],'indexofpixel':class1_sample.iloc[0]['indexofpixel'],'sourceofpixel':class1_sample.iloc[0]['sourceofpixel']}
class2_sample= {'feature0':class2_sample.iloc[0]['feature0'],'feature1':class2_sample.iloc[0]['feature1'],'feature2':class2_sample.iloc[0]['feature2'],'feature3':class2_sample.iloc[0]['feature3'],'feature4':class2_sample.iloc[0]['feature4'],'label':class2_sample.iloc[0]['label'],'indexofpixel':class2_sample.iloc[0]['indexofpixel'],'sourceofpixel':class2_sample.iloc[0]['sourceofpixel']}
newStore.append(class1_sample)
newStore.append(class2_sample)

In [124]:
a=np.ones(1568)
a=a+1
a

array([ 2.,  2.,  2., ...,  2.,  2.,  2.])

In [ ]:
start=time.time()
count=0
while newStore:
    count = count+1
    oldlength = len(Store)
    for element in newStore:
        Store.append(element)
    
    temp=pd.DataFrame(data=newStore)
    temp=np.array(temp)
    temp=temp[:,0:5]
    np.cast[float](temp)

    train_temp=np.array(train)
    train_temp=train_temp[:,0:5]
    
    start1=time.time()
    #dis=distance_matrix(temp.reshape(len(newStore),5),train_temp)
    dis=distance.cdist(temp.reshape(len(newStore),5),train_temp,'euclidean')
    nearest_q_matrix=dis.min(0)
    nearest_q_indexmat=dis.argmin(axis=0)
    print("executing distance calculation time:%5.1f minute"%((time.time()-start1)/60))
    
    #update the nearest distance of q in (T-S) dataset, and the corresponding p in S.
    if count==1:
        nearest_q={'p_index':nearest_q_indexmat,'nearest_dis':nearest_q_matrix}
        nearest_q=pd.DataFrame(data=nearest_q)
        nearest_q=pd.concat([nearest_q,train['label']],axis=1,join='outer')
    else:
        nearest_q_indexmat=nearest_q_indexmat+oldlength
        nearest_q_new={'p_index_new':nearest_q_indexmat,'nearest_dis_new':nearest_q_matrix}
        nearest_q_new=pd.DataFrame(data=nearest_q_new)
        nearest_q=pd.concat([nearest_q,nearest_q_new],axis=1,join='outer')
        nearest_q['p_index'] = np.where((nearest_q['nearest_dis_new']<nearest_q['nearest_dis']),nearest_q['p_index_new'],nearest_q['p_index'])
        nearest_q['nearest_dis'] = np.where((nearest_q['nearest_dis_new']<nearest_q['nearest_dis']),nearest_q['nearest_dis_new'],nearest_q['nearest_dis'])
        nearest_q=nearest_q.drop('p_index_new',axis=1)
        nearest_q=nearest_q.drop('nearest_dis_new',axis=1)
        #print (nearest_q)
    
    Store_dataframe=pd.DataFrame(data=Store)
    
    #attach p's label to the dataframe of nearest_q
    p_label=pd.DataFrame(Store_dataframe.iloc[nearest_q['p_index']]['label'])
    p_label.columns=['p_label']
    p_label=pd.DataFrame.reset_index(p_label).drop('index',axis=1)
    nearest_q=pd.concat([nearest_q,p_label],axis=1,join='outer')
    
    #choose those data  q whose label is not equal to the lapel of nearest_q
    # and then creat the dataframe called different_predict_points
    temp_setting=np.ones(len(train))
    temp_setting=pd.DataFrame(data=temp_setting)
    temp_setting.columns=['setting']
    nearest_q=pd.concat([nearest_q,temp_setting],axis=1,join='outer')
    nearest_q['choose'] = np.where((nearest_q['label']!=nearest_q['p_label']),nearest_q['setting'],np.nan)
    
    different_predic_points=pd.DataFrame(nearest_q[nearest_q['choose']==1])
    
    nearest_q=nearest_q.drop('setting',axis=1)
    nearest_q=nearest_q.drop('choose',axis=1)
    
    
    newStore=list()
 
    start2=time.time()
    for i in range(len(Store)):
        temp=different_predic_points[different_predic_points['p_index']==i]
        temp=pd.DataFrame.reset_index(temp)
        if len(temp)>0 :
            location=temp['nearest_dis'].idxmin(axis=1)
            add_positon=int(temp.loc[location]['index'])
            temp_newone= {'feature0':train.loc[add_positon]['feature0'],'feature1':train.loc[add_positon]['feature1'],'feature2':train.loc[add_positon]['feature2'],'feature3':train.loc[add_positon]['feature3'],'feature4':train.loc[add_positon]['feature4'],'label':train.loc[add_positon]['label'],'indexofpixel':train.loc[add_positon]['indexofpixel'],'sourceofpixel':train.loc[add_positon]['sourceofpixel']}
            newStore.append(temp_newone)
            train=train.drop(add_positon)
            nearest_q=nearest_q.drop(add_positon)
            #print ("train size:%d"%(len(train)))
            #print ("nearest size:%d"%(len(nearest_q))) 
            #print (nearest_q)
    print("executing adding new ones time:%5.1f minute"%((time.time()-start2)/60))
    
    nearest_q=nearest_q.drop('p_label',axis=1)
    train=pd.DataFrame.reset_index(train).drop('index',axis=1)
    nearest_q=pd.DataFrame.reset_index(nearest_q).drop('index',axis=1)
    print("current count:%d current size of store:%d time:%5.1f minute"%(count,len(Store),(time.time()-start)/60))

executing distance calculation time:  0.0 minute
executing adding new ones time:  0.0 minute
current count:1 current size of store:2 time:  0.0 minute
executing distance calculation time:  0.0 minute
executing adding new ones time:  0.0 minute
current count:2 current size of store:4 time:  0.0 minute
executing distance calculation time:  0.0 minute
executing adding new ones time:  0.0 minute
current count:3 current size of store:8 time:  0.0 minute
executing distance calculation time:  0.0 minute
executing adding new ones time:  0.0 minute
current count:4 current size of store:16 time:  0.1 minute
executing distance calculation time:  0.0 minute
executing adding new ones time:  0.0 minute
current count:5 current size of store:32 time:  0.1 minute
executing distance calculation time:  0.0 minute
executing adding new ones time:  0.1 minute
current count:6 current size of store:63 time:  0.2 minute
executing distance calculation time:  0.0 minute
executing adding new ones time:  0.1 minut

In [ ]:
start=time.time()
count=0
while newStore:
    count = count+1
    for element in newStore:
        Store.append(element)    
    
    temp=pd.DataFrame(data=Store)
    temp=np.array(temp)
    temp=temp[:,0:5]
    np.cast[float](temp)

    train_temp=np.array(train)
    train_temp=train_temp[:,0:5]

    rep = np.zeros([len(Store),2])
    rep[:,1]=100

    for i in range(len(train_temp)):
        #if i%10000==0:
            #print("current executing:%d time:%5.1f minute"%(i,(time.time()-start)/60))
        distance_matrix=distance.cdist(temp.reshape(len(Store),5),train_temp[i].reshape(1,5),metric='euclidean')
        nearest=np.amin(distance_matrix)
        indice=np.where(distance_matrix==nearest)
        #print(distance_matrix)
        #print(indice)
        #print(indice[0][0])
        if int(Store[indice[0][0]]['label'])!=int(train.iloc[i]['label']):
            if nearest<rep[indice[0][0],1]:
                rep[indice[0][0],0]=i
                rep[indice[0][0],1]=nearest
            else:
                pass
        else:
            pass   
    #print("current executing:%d time:%5.1f minute"%(i,(time.time()-start)/60))

    newStore=list()
    #print("store")
    #print (Store)

    for i in range(len(rep)):
        if rep[i][1] != 100:
            loc = int(rep[i][0])
            #print (loc)
            temp_newone= {'feature0':train.loc[loc]['feature0'],'feature1':train.loc[loc]['feature1'],'feature2':train.loc[loc]['feature2'],'feature3':train.loc[loc]['feature3'],'feature4':train.loc[loc]['feature4'],'label':train.loc[loc]['label'],'indexofpixel':train.loc[loc]['indexofpixel'],'sourceofpixel':train.loc[loc]['sourceofpixel']}
            newStore.append(temp_newone)
            train=train.drop(loc)
    train=pd.DataFrame.reset_index(train).drop('index',axis=1)  

    print("current count:%d current size of store:%d time:%5.1f minute"%(count,len(Store),(time.time()-start)/60))
    